NHS Digital produce ["business rules"](https://digital.nhs.uk/data-and-information/data-collections-and-data-sets/data-collections/quality-and-outcomes-framework-qof/quality-and-outcome-framework-qof-business-rules/quality-and-outcomes-framework-qof-business-rules-v42-2019-2020-baseline-release) to support the correct identification and counts of patients to support General Practice reimbursement under the quality and outcomes framework. They produce a word document which in turn links to a "ref set" that can be interrogated on the NHS SnoMed browser.

Here we set out to interrograte the AF QoF [business rules refset for medicines](https://termbrowser.nhs.uk/?perspective=full&conceptId1=12463501000001109&edition=uk-edition&release=v20190601&server=https://termbrowser.nhs.uk/sct-browser-api/snomed&langRefset=999001261000000100,999000691000001104) against the NHS dm+d for discrepancies. The refset has an id number of 12463501000001109 and has "188 members"

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts

In [2]:
##First we identify VMPs.
sql = '''SELECT
  *
FROM
  `ebmdatalab.dmd.vmp`
WHERE
  bnf_code LIKE "0208020Y0%" ##RIVAROXABAN
  OR bnf_code LIKE "0208020X0%" ##DABIGATRAN
  OR bnf_code LIKE "0208020Z0%" ##APIXABAN
  OR bnf_code LIKE "0208020AA%" ##EDOXABAN
  OR bnf_code LIKE "0208020V0%" ##WARFARIN
  OR bnf_code LIKE "0208020H0%" ##ACENOCOUMARLO
  OR bnf_code LIKE "0208020N0%" ###PHENIDIONE'''
    
vmp_anticoag = bq.cached_read(sql, csv_path='vmp_anticoagp.csv')
vmp_anticoag.head(10)

,id,vpiddt,vpidprev,vtm,invalid,nm,abbrevnm,basis,nmdt,nmprev,...,glu_f,pres_f,cfc_f,non_avail,non_availdt,df_ind,udfs,udfs_uom,unit_dose_uom,bnf_code
0,8797911000001107,NaN,NaN,48603004,False,Warfarin 10mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAVAV
1,8798011000001109,NaN,NaN,48603004,False,Warfarin 10mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAVAV
2,8798511000001101,NaN,NaN,48603004,False,Warfarin 3mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAMAM
3,8798611000001102,NaN,NaN,48603004,False,Warfarin 3mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAMAM
4,8798711000001106,NaN,NaN,48603004,False,Warfarin 5mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAIAI
5,8798811000001103,NaN,NaN,48603004,False,Warfarin 5mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAIAI
6,8798111000001105,NaN,NaN,48603004,False,Warfarin 1mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAANAN
7,8798211000001104,NaN,NaN,48603004,False,Warfarin 1mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAANAN
8,8798311000001107,NaN,NaN,48603004,False,Warfarin 2mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAJAJ
9,8798411000001100,NaN,NaN,48603004,False,Warfarin 2mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAJAJ


In [3]:
vmp_anticoag.count()

id               43
vpiddt            2
vpidprev          2
vtm              43
invalid          43
nm               43
abbrevnm          0
basis            43
nmdt              1
nmprev            1
basis_prev        1
nmchange          1
combprod          1
pres_stat        43
sug_f            43
glu_f            43
pres_f           43
cfc_f            43
non_avail         1
non_availdt       1
df_ind           43
udfs             24
udfs_uom         24
unit_dose_uom    24
bnf_code         43
dtype: int64

In [4]:
vmp_anticoag.nunique(axis = 0)

id               43
vpiddt            1
vpidprev          2
vtm               7
invalid           1
nm               43
abbrevnm          0
basis             2
nmdt              1
nmprev            1
basis_prev        1
nmchange          1
combprod          1
pres_stat         1
sug_f             2
glu_f             1
pres_f            1
cfc_f             1
non_avail         1
non_availdt       1
df_ind            3
udfs              1
udfs_uom          2
unit_dose_uom     2
bnf_code         38
dtype: int64

The VMP dmd file give us 43 products of which none are invalid......

Now we mst look for AMPs

In [7]:
sql = '''SELECT
  *
FROM
  `ebmdatalab.dmd.amp`
WHERE
  bnf_code LIKE "0208020Y0%" ##RIVAROXABAN
  OR bnf_code LIKE "0208020X0%" ##DABIGATRAN
  OR bnf_code LIKE "0208020Z0%" ##APIXABAN
  OR bnf_code LIKE "0208020AA%" ##EDOXABAN
  OR bnf_code LIKE "0208020V0%" ##WARFARIN
  OR bnf_code LIKE "0208020H0%" ##ACENOCOUMARLO
  OR bnf_code LIKE "0208020N0%" ###PHENIDIONE
  '''
    
amp_anticoag = bq.cached_read(sql, csv_path='amp_anticoagp.csv')
amp_anticoag.head(10)

,id,invalid,vmp,nm,abbrevnm,descr,nmdt,nm_prev,supp,lic_auth,lic_auth_prev,lic_authchange,lic_authchangedt,combprod,flavour,ema,parallel_import,avail_restrict,bnf_code
0,29902111000001100,False,29903211000001100,Lixiana 15mg tablets,None,Lixiana 15mg tablets (Daiichi Sankyo UK Ltd),NaT,None,2081301000001107,1,None,None,NaT,NaN,None,True,False,1,0208020AABBAAAA
1,29902411000001105,False,29903311000001108,Lixiana 30mg tablets,None,Lixiana 30mg tablets (Daiichi Sankyo UK Ltd),NaT,None,2081301000001107,1,None,None,NaT,NaN,None,True,False,1,0208020AABBABAB
2,29902711000001104,False,29903411000001101,Lixiana 60mg tablets,None,Lixiana 60mg tablets (Daiichi Sankyo UK Ltd),NaT,None,2081301000001107,1,None,None,NaT,NaN,None,True,False,1,0208020AABBACAC
3,738511000001103,False,319740004,Sinthrome 1mg tablets,None,Sinthrome 1mg tablets (Merus Labs Luxco II S.a...,NaT,None,22480511000001101,1,None,None,NaT,NaN,None,False,False,1,0208020H0BBAAAA
4,3781211000001108,False,319745009,Phenindione 10mg tablets,None,Phenindione 10mg tablets (Advanz Pharma),NaT,None,24129411000001106,1,None,None,NaT,NaN,None,False,False,1,0208020N0AAAAAA
5,3781411000001107,False,319745009,Phenindione 10mg tablets,None,Phenindione 10mg tablets (Alliance Healthcare ...,NaT,None,2089901000001107,1,None,None,NaT,NaN,None,False,False,1,0208020N0AAAAAA
6,3782311000001109,False,319746005,Phenindione 25mg tablets,None,Phenindione 25mg tablets (Advanz Pharma),NaT,None,24129411000001106,1,None,None,NaT,NaN,None,False,False,1,0208020N0AAABAB
7,3782511000001103,False,319746005,Phenindione 25mg tablets,None,Phenindione 25mg tablets (Alliance Healthcare ...,NaT,None,2089901000001107,1,None,None,NaT,NaN,None,False,False,1,0208020N0AAABAB
8,421411000001103,False,319733000,Warfarin 1mg tablets,None,Warfarin 1mg tablets (A A H Pharmaceuticals Ltd),NaT,None,3144701000001104,1,None,None,NaT,NaN,None,False,False,1,0208020V0AAAAAA
9,303111000001107,False,319733000,Warfarin 1mg tablets,None,Warfarin 1mg tablets (Actavis UK Ltd),NaT,None,3875201000001104,1,None,None,NaT,NaN,None,False,False,1,0208020V0AAAAAA


In [8]:
amp_anticoag.count()

id                  128
invalid             128
vmp                 128
nm                  128
abbrevnm              0
descr               128
nmdt                  0
nm_prev               0
supp                128
lic_auth            128
lic_auth_prev         0
lic_authchange        0
lic_authchangedt      0
combprod              1
flavour               0
ema                 128
parallel_import     128
avail_restrict      128
bnf_code            128
dtype: int64

In [9]:
amp_anticoag.nunique(axis = 0)

id                  128
invalid               1
vmp                  43
nm                   47
abbrevnm              0
descr               128
nmdt                  0
nm_prev               0
supp                 31
lic_auth              2
lic_auth_prev         0
lic_authchange        0
lic_authchangedt      0
combprod              1
flavour               0
ema                   2
parallel_import       1
avail_restrict        4
bnf_code             42
dtype: int64